In [1]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications import VGG16, preprocess_input, decode_predictions
import cv2

# Load the VGG16 model pre-trained on ImageNet
model = VGG16(weights='imagenet')
model.summary()

# Load an input image and preprocess it
img_path = 'elephant.jpg'  # Replace with your own image path
img = image.load_img(img_path, target_size=(224, 224))
x = image.img_to_array(img)
x = np.expand_dims(x, axis=0)
x = preprocess_input(x)

### Part 1: Visualizing ConvNet Filters ###
# Extract the filters of the first Conv layer
layer_name = 'block1_conv1'
filters, biases = model.get_layer(name=layer_name).get_weights()

# Normalize filter values for better visualization
filters = (filters - filters.min()) / (filters.max() - filters.min())

# Plotting the filters
n_filters = 6  # Number of filters to visualize
fig, axes = plt.subplots(1, n_filters, figsize=(20, 20))
for i in range(n_filters):
    # Get the ith filter
    f = filters[:, :, :, i]
    # Plot each filter
    axes[i].imshow(f[:, :, 0], cmap='gray')  # Visualizing in grayscale
    axes[i].axis('off')
plt.suptitle('ConvNet Filters')
plt.show()

### Part 2: Visualizing Intermediate Activations ###
# Create a new model that outputs the activations of the first 8 layers
layer_outputs = [layer.output for layer in model.layers[:8]]
activation_model = Model(inputs=model.input, outputs=layer_outputs)

# Pass the image through the model
activations = activation_model.predict(x)

# Visualize the activations for a specific layer
layer_names = [layer.name for layer in model.layers[:8]]
layer_idx = 1  # Visualize activations of the second layer
layer_activation = activations[layer_idx]

# Plotting the activations
n_filters = 6  # Number of filters to visualize
fig, axes = plt.subplots(1, n_filters, figsize=(20, 20))
for i in range(n_filters):
    # Get the ith activation map
    activation_map = layer_activation[0, :, :, i]
    axes[i].imshow(activation_map, cmap='viridis')
    axes[i].axis('off')
plt.suptitle('Intermediate Activations')
plt.show()

### Part 3: Visualizing Heatmaps (Grad-CAM) ###
# Function to compute Grad-CAM
def compute_gradcam(model, img_array, layer_name, class_idx):
    grad_model = Model([model.inputs], [model.get_layer(layer_name).output, model.output])
    with tf.GradientTape() as tape:
        conv_outputs, predictions = grad_model(img_array)
        loss = predictions[:, class_idx]
        grads = tape.gradient(loss, conv_outputs)
        pooled_grads = tf.reduce_mean(grads, axis=(0, 1, 2))
    conv_outputs = conv_outputs.numpy()[0]
    pooled_grads = pooled_grads.numpy()
    for i in range(pooled_grads.shape[-1]):
        conv_outputs[:, :, i] *= pooled_grads[i]
    heatmap = np.mean(conv_outputs, axis=-1)
    heatmap = np.maximum(heatmap, 0)
    heatmap /= np.max(heatmap)
    return heatmap

# Example to compute Grad-CAM for the 'block5_conv3' layer for class index 386 (elephant)
heatmap = compute_gradcam(model, x, 'block5_conv3', 386)

# Plot the heatmap
plt.matshow(heatmap)
plt.suptitle('Grad-CAM Heatmap')
plt.show()

# Superimpose the heatmap on the original image
img = cv2.imread(img_path)
heatmap = cv2.resize(heatmap, (img.shape[1], img.shape[0]))
heatmap = np.uint8(255 * heatmap)
heatmap = cv2.applyColorMap(heatmap, cv2.COLORMAP_JET)
superimposed_img = heatmap * 0.4 + img

# Save and display the superimposed image
cv2.imwrite('elephant_cam.jpg', superimposed_img)
superimposed_img_rgb = cv2.cvtColor(superimposed_img, cv2.COLOR_BGR2RGB)
plt.imshow(superimposed_img_rgb)
plt.axis('off')
plt.suptitle('Elephant with Grad-CAM Heatmap')
plt.show()


KeyboardInterrupt: 